# Librairies

In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Chargement de données

In [11]:
train_base_dir = '../Data/Data/train/'
clear_dir = os.path.join(train_base_dir, 'clear')
classes = ['clear', 'distorsion1', 'distorsion2', 'distorsion3','mvt1','mvt2','mvt3','gauss1','gauss2','gauss3','pixel1','pixel2','pixel3'] 

def verifier_et_lister_contenu(repertoire):
    if os.path.exists(repertoire):
        print(f"Le dossier '{repertoire}' contient les fichiers suivants :")
        for filename in os.listdir(repertoire):
            print(filename)
    else:
        print(f"Le dossier '{repertoire}' n'existe pas. Vérifiez le chemin.")

verifier_et_lister_contenu(clear_dir)
for class_name in classes[1:]:
    verifier_et_lister_contenu(os.path.join(train_base_dir, class_name))

image_size = (224, 224)
batch_size = 64  


def charger_et_preparer_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  
    img = tf.image.resize(img, image_size)  
    img = img / 255.0  
    return img


def charger_images_et_labels(repertoire, label):
    images_paths = [os.path.join(repertoire, filename) for filename in os.listdir(repertoire)]
    dataset = tf.data.Dataset.from_tensor_slices(images_paths)
    dataset = dataset.map(lambda img_path: (charger_et_preparer_image(img_path), label), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

clear_dataset = charger_images_et_labels(clear_dir, 0)


Le dossier '../Data/Data/train/clear' contient les fichiers suivants :
clear_34.jpg
clear_172.jpg
clear_1576.jpg
clear_1107.jpg
clear_38.jpg
clear_848.jpg
clear_1239.jpg
clear_992.jpg
clear_1462.jpg
clear_1297.jpg
clear_495.jpg
clear_259.jpg
clear_132.jpg
clear_62.jpg
clear_1096.jpg
clear_116.jpg
clear_1321.jpg
clear_286.jpg
clear_143.jpg
clear_1608.jpg
clear_1413.jpg
clear_847.jpg
clear_905.jpg
clear_190.jpg
clear_741.jpg
clear_511.jpg
clear_993.jpg
clear_1027.jpg
clear_391.jpg
clear_348.jpg
clear_770.jpg
clear_1482.jpg
clear_820.jpg
clear_1588.jpg
clear_1129.jpg
clear_937.jpg
clear_56.jpg
clear_367.jpg
clear_1318.jpg
clear_110.jpg
clear_1517.jpg
clear_1366.jpg
clear_1523.jpg
clear_201.jpg
clear_1403.jpg
clear_346.jpg
clear_600.jpg
clear_831.jpg
clear_506.jpg
clear_1190.jpg
clear_284.jpg
clear_1565.jpg
clear_1052.jpg
clear_556.jpg
clear_1474.jpg
clear_711.jpg
clear_661.jpg
clear_25.jpg
clear_1180.jpg
clear_35.jpg
clear_386.jpg
clear_48.jpg
clear_1460.jpg
clear_1323.jpg
clear_200.jpg
c

# Création du modèle

In [14]:
model = keras.models.Sequential([
    keras.layers.Input((224, 224, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(2, activation='softmax')  # 2 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Distorsion

In [ ]:
# DIstorsion1

distorsion1_dir = os.path.join(train_base_dir, 'distorsion1')
distorsion_dataset = charger_images_et_labels(distorsion1_dir, 1)

full_dataset = clear_dataset.concatenate(distorsion_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git




Total d'images dans clear_dataset: 3225
Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 40s 917ms/step - accuracy: 0.9437 - loss: 12.9114 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 37s 890ms/step - accuracy: 0.4376 - loss: 10.4080 - val_accuracy: 1.0000 - val_loss: 1.9153e-04
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 37s 891ms/step - accuracy: 0.4618 - loss: 2.3719 - val_accuracy: 0.0046 - val_loss: 0.7022
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 36s 872ms/step - accuracy: 0.8390 - loss: 0.6897 - val_accuracy: 0.0199 - val_loss: 0.7061
Epoch 5/10
26/41 ━━━━━━━━━━━━━━━━━━━━ 12s 821ms/step - accuracy: 0.9969 - loss: 0.3979

2024-11-11 15:41:59.807574: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 38535424 bytes after encountering the first element of size 38535424 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 814ms/step - accuracy: 0.9376 - loss: 5.3332

2024-11-11 15:42:12.305876: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 38535424 bytes after encountering the first element of size 38535424 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


41/41 ━━━━━━━━━━━━━━━━━━━━ 37s 896ms/step - accuracy: 0.9349 - loss: 5.5073 - val_accuracy: 1.0000 - val_loss: 1.4509e-07
Epoch 6/10
26/41 ━━━━━━━━━━━━━━━━━━━━ 12s 835ms/step - accuracy: 0.4458 - loss: 5.5392

...

2024-11-11 15:43:14.300091: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 38535424 bytes after encountering the first element of size 38535424 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 818ms/step - accuracy: 0.3917 - loss: 0.6994

2024-11-11 15:43:27.473821: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 38535424 bytes after encountering the first element of size 38535424 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


41/41 ━━━━━━━━━━━━━━━━━━━━ 37s 904ms/step - accuracy: 0.3896 - loss: 0.6995 - val_accuracy: 1.0000 - val_loss: 0.6768
Epoch 8/10
11/41 ━━━━━━━━━━━━━━━━━━━━ 25s 835ms/step - accuracy: 0.0260 - loss: 0.7091   

KeyboardInterrupt: 

In [ ]:
# Distorsion2

distorsion2_dir = os.path.join(train_base_dir, 'distorsion2')
distorsion_dataset = charger_images_et_labels(distorsion2_dir, 1)

full_dataset = clear_dataset.concatenate(distorsion_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Distorsion3

distorsion3_dir = os.path.join(train_base_dir, 'distorsion3')
distorsion_dataset = charger_images_et_labels(distorsion3_dir, 1)

full_dataset = clear_dataset.concatenate(distorsion_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





# Flou Mouvement

In [ ]:
# Mvt1

mvt1_dir = os.path.join(train_base_dir, 'mvt1')
mvt_dataset = charger_images_et_labels(mvt1_dir, 1)

full_dataset = clear_dataset.concatenate(mvt_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Mvt2

mvt2_dir = os.path.join(train_base_dir, 'mvt2')
mvt_dataset = charger_images_et_labels(mvt2_dir, 1)

full_dataset = clear_dataset.concatenate(mvt_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Mvt3

mvt3_dir = os.path.join(train_base_dir, 'mvt3')
mvt_dataset = charger_images_et_labels(mvt3_dir, 1)

full_dataset = clear_dataset.concatenate(mvt_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





# Flou gaussien

In [ ]:
# Gauss1

gauss1_dir = os.path.join(train_base_dir, 'gauss1')
gauss_dataset = charger_images_et_labels(gauss1_dir, 1)

full_dataset = clear_dataset.concatenate(gauss_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Gauss2

gauss2_dir = os.path.join(train_base_dir, 'gauss2')
gauss_dataset = charger_images_et_labels(gauss2_dir, 1)

full_dataset = clear_dataset.concatenate(gauss_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Gauss3

gauss3_dir = os.path.join(train_base_dir, 'gauss3')
gauss_dataset = charger_images_et_labels(gauss3_dir, 1)

full_dataset = clear_dataset.concatenate(gauss_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





# Pixelisation

In [ ]:
# Pixel1

pixel1_dir = os.path.join(train_base_dir, 'pixel1')
pixel_dataset = charger_images_et_labels(pixel1_dir, 1)

full_dataset = clear_dataset.concatenate(pixel_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Pixel2

pixel2_dir = os.path.join(train_base_dir, 'pixel2')
pixel_dataset = charger_images_et_labels(pixel2_dir, 1)

full_dataset = clear_dataset.concatenate(pixel_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git





In [ ]:
# Pixel3

pixel3_dir = os.path.join(train_base_dir, 'pixel3')
pixel_dataset = charger_images_et_labels(pixel3_dir, 1)

full_dataset = clear_dataset.concatenate(pixel_dataset)

total_images = 0
for images, labels in full_dataset:
    total_images += images.shape[0]

print(f"Total d'images dans clear_dataset: {total_images}")

train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

history = model.fit(
    train_dataset, 
    validation_data=validation_dataset,  
    epochs=10,   
    verbose=1    
)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

predictions = model.predict(validation_dataset)
predictions = np.argmax(predictions, axis=1)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)

cm = confusion_matrix(true_labels, predictions)
class_names = [str(i) for i in range(2)]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.savefig("matrice.png")
plt.show()

# Sauvegarde du modèle
# model.save('my_model.keras') #modele lourd pour push sur git



